Used tutorial https://github.com/maobedkova/TopicModelling_PySpark_SparkNLP/blob/master/Topic_Modelling_with_PySpark_and_Spark_NLP.ipynb for reference

In [1]:
import sparknlp
from sparknlp.annotator import Tokenizer, PerceptronModel
from sparknlp.base import DocumentAssembler
from pyspark.ml import Pipeline
from sparknlp.annotator import *
from sparknlp.base import *
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF
from pyspark.ml.clustering import LDA
from sparknlp.annotator import StopWordsCleaner
from pyspark.sql import types as T
from pyspark.sql import functions as F
from sparknlp.annotator import NGramGenerator
from sparknlp.base import Finisher
from pyspark.ml.tuning import ParamGridBuilder

spark = sparknlp.start()

24/11/20 20:15:28 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
data = spark.read.csv("../data_topicmodel.csv", header= True).select(["id", "cleaned_text"])

## Preprocessing

In [3]:
documentAssembler = DocumentAssembler()\
     .setInputCol("cleaned_text")\
     .setOutputCol('document')

In [4]:
tokenizer = Tokenizer() \
            .setInputCols(['document'])\
            .setOutputCol('tokenized')

In [5]:
normalizer = Normalizer() \
     .setInputCols(['tokenized']) \
     .setOutputCol('normalized') 

In [6]:
english = [
    "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", 
    "because", "been", "before", "being", "below", "between", "both", "but", "by", "can", "cannot", "could", "did", 
    "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", 
    "he", "her", "here", "hers", "herself", "him", "himself", "his", "how", "i", "if", "in", "into", "is", "it", 
    "its", "itself", "let", "me", "more", "most", "must", "my", "myself", "no", "nor", "not", "of", "off", "on", 
    "once", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "some", "such", 
    "than", "that", "the", "their", "theirs", "them", "themselves", "then", "there", "these", "they", "this", "those", 
    "through", "to", "too", "under", "until", "up", "very", "was", "we", "were", "what", "when", "where", "which", 
    "while", "who", "whom", "why", "with", "would", "you", "your", "yours", "yourself", "yourselves", "will", "ll", 
    "re", "ve", "d", "s", "m", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", 
    "s", "t", "u", "v", "w", "x", "y", "z", "many", "us", "ok", "hows", "ive", "ill", "im", "cant", "topics", "topic",
    "discuss", "thoughts", "yo", "thats", "whats", "lets", "nothing", "oh", "omg", 
         "things", "stuff", "yall", "haha", "yes", "no", "wo", "like", 'good', 
         'work', 'got', 'going', 'dont', 'really', 'want', 'make', 'think', 
         'know', 'feel', 'people', 'life', "getting", "lot" "great", "i", "me", 
         "my", "myself", "we", "our", "ours", "ourselves", 
        "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", 
        "himself", "she", "her", "hers", "herself", "it", "its", "itself", 
        "they", "them", "their", "theirs","themselves", "what", "which", "who", 
        "whom", "this", "that", "these", "those", "am", "is", "are", "was", 
        "were", "be", "been", "being", "have", "has", "had", "having", "do", 
        "does", "did", "doing", "will", "would", "should", "can", "could", "may",
        "might", "must", "shall", "ought", "about", "above", "across", "after", 
        "against", "along", "amid", "among", "around", "as", "at", "before", "behind",
        "below", "beneath", "beside", "between", "beyond", "but", "by", 
        "concerning", "considering", "despite", "down", "during", "except", "for",
        "from", "in", "inside", "into", "like", "near", "next", "notwithstanding",
        "of", "off", "on", "onto", "opposite", "out", "outside", "over", "past",
        "regarding", "round", "since", "than", "through", "throughout", "till", 
        "to", "toward", "towards", "under", "underneath", "unlike", "until", "up",
        "upon", "versus", "via", "with", "within", "without", "cant", "cannot", 
        "couldve", "couldnt", "didnt", "doesnt", "dont", "hadnt", "hasnt", 
        "havent", "hed", "hell", "hes", "howd", "howll", "hows", "id", "ill", 
        "im", "ive", "isnt", "itd", "itll", "its", "lets", "mightve", "mustve", 
        "mustnt", "shant", "shed", "shell", "shes", "shouldve", "shouldnt", 
        "thatll", "thats", "thered", "therell", "therere", "theres", "theyd", 
        "theyll", "theyre", "theyve", "wed", "well", "were", "weve", "werent", 
        "whatd", "whatll", "whatre", "whats", "whatve", "whend", "whenll", 
        "whens", "whered", "wherell", "wheres", "whichd", "whichll", "whichre", 
        "whichs", "whod", "wholl", "whore", "whos", "whove", "whyd", "whyll", 
        "whys", "wont", "wouldve", "wouldnt", "youd", "youll", "youre", "youve",
        "f", "m", "because", "go", "lot", "get", "still", "way", "something", "much",
        "thing", "someone", "person", "anything", "goes", "ok", "so", "just", "mostly", 
        "put", "also", "lots", "yet", "ha", "etc"]

time = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", 
        "sunday", "morning", "noon", "afternoon", "evening", "night", "midnight",
        "dawn", "dusk", "week", "weekend", "weekends","weekly", "today", 
        "yesterday", "tomorrow", "yesterdays", "todays", "mondays", "tuesdays",
        "wednesdays", "thursdays", "fridays", "saturdays", "sundays", "day",
        "everyday", "daily", "workday", 'time', 'month', 'year', 'pm', 'am', "ago",
        "year"]

reddit = ["welcome", "hi", "hello", "sub", "reddit", "thanks", "thank", "maybe",
          "wo30", "mods", "mod", "moderators", "subreddit", "btw", "aw", "aww", 
          "aww", "hey", "hello", "join", "joined", "post", "rselfimprovement"]

topic_specific = ["self", "improvement", "change", "action",
    'change', 'start', 'goal', 'habit', 'new', 'old', 
    'care', 'world', 'everyone', 'love', 'u', 'right', 'mean', 'matter',
    'best', 'step', 'focus', 'hard', 'small',
    'bad', 'help', 'time', 'problem', 'issue', 'advice',
    'bit', 'experience', 'different',
    'point', 'situation', 'negative', 'control', 'positive',
    'use', 'question', 'idea', 'amp', 'medium', 'hour', 'day', 'minute',
    'aaaaloot', "selfimprovement", "_"]

stopwords = english + time + reddit + topic_specific

from sparknlp.annotator import StopWordsCleaner

stopwords_cleaner = StopWordsCleaner() \
     .setInputCols(['normalized']) \
     .setOutputCol('unigrams') \
     .setStopWords(stopwords)

In [7]:
#from sparknlp.annotator import NGramGenerator

#ngrammer = NGramGenerator() \
 #   .setInputCols(['normalized']) \
  #  .setOutputCol('ngrams') \
   # .setN(3) \
    #.setEnableCumulative(True) \
    #.setDelimiter('_')

In [8]:
pos = PerceptronModel.load("/project/macs40123/spark-jars/pos_anc_en_3.0.0_3.0_1614962126490/")\
      .setInputCols("document", "unigrams")\
      .setOutputCol("pos")

In [9]:
from sparknlp.base import Finisher

finisher = Finisher().setInputCols(['unigrams', 'pos'])

In [10]:
my_pipeline = Pipeline(
      stages = [
          documentAssembler,
          tokenizer,
          normalizer,
          stopwords_cleaner,
          #ngrammer,
          pos,
          finisher
      ])

In [11]:
pipelineModel = my_pipeline.fit(data)
processed_data = pipelineModel.transform(data)
processed_data.show(1)

+-----+--------------------+--------------------+--------------------+
|   id|        cleaned_text|   finished_unigrams|        finished_pos|
+-----+--------------------+--------------------+--------------------+
|hk5r2|i had an appointm...|[appointment, den...|[NN, NN, JJ, CD, ...|
+-----+--------------------+--------------------+--------------------+
only showing top 1 row



In [12]:
processed_data.columns

['id', 'cleaned_text', 'finished_unigrams', 'finished_pos']

In [13]:
processed_data.select('finished_unigrams').show(10, truncate = 100)

+----------------------------------------------------------------------------------------------------+
|                                                                                   finished_unigrams|
+----------------------------------------------------------------------------------------------------+
|[appointment, dentist, last, one, reminder, email, two, reminder, phone, calls, marked, google, c...|
|[created, site, several, months, finished, decided, release, beta, form, rather, simply, stagnate...|
|[recently, took, made, changes, overall, look, hope, pleasing, enough, vision, place, share, info...|
|[grew, body, dysmorphia, eating, disorders, social, anixety, depression, early, years, remember, ...|
|[ask, content, never, process, stop, made, speak, months, hopeless, yearold, drunk, regular, frie...|
|[opportunity, exists, feeling, opportunities, recharge, motivation, build, selfconfidence, affirm...|
|[comfort, zone, friends, ton, probably, hit, bar, club, never, comfortab

In [14]:
processed_data.show(3)

+-----+--------------------+--------------------+--------------------+
|   id|        cleaned_text|   finished_unigrams|        finished_pos|
+-----+--------------------+--------------------+--------------------+
|hk5r2|i had an appointm...|[appointment, den...|[NN, NN, JJ, CD, ...|
|iqimz|i created this si...|[created, site, s...|[VBN, NN, JJ, NNS...|
|pfzt5|hello everyone  i...|[recently, took, ...|[RB, VBD, VBN, NN...|
+-----+--------------------+--------------------+--------------------+
only showing top 3 rows



In [15]:
processed_data.persist()

DataFrame[id: string, cleaned_text: string, finished_unigrams: array<string>, finished_pos: array<string>]

In [16]:
def filter_unigrams(finished_unigrams, finished_pos):
    '''Filters individual words based on their POS tag'''
    return [word for word, pos in zip(finished_unigrams, finished_pos)
            if pos in ['JJ', 'NN', 'NNS', 'NNPS']]

udf_filter_unigrams = F.udf(filter_unigrams, T.ArrayType(T.StringType()))

In [17]:
processed_data = processed_data.withColumn('filtered_unigrams_by_pos', udf_filter_unigrams(
                                                   F.col('finished_unigrams'),
                                                   F.col('finished_pos')))

processed_data.select("filtered_unigrams_by_pos").show(10, truncate=100)

+----------------------------------------------------------------------------------------------------+
|                                                                            filtered_unigrams_by_pos|
+----------------------------------------------------------------------------------------------------+
|[appointment, dentist, last, reminder, email, reminder, phone, calls, google, calendar, glance, t...|
|[site, several, months, release, beta, form, forgive, useful, community, form, direction, future,...|
|[changes, hope, vision, place, share, information, mindset, prosperity, mindset, money, purpose, ...|
|[body, dysmorphia, disorders, social, anixety, depression, early, years, fat, weight, case, healt...|
|[content, process, stop, speak, months, hopeless, yearold, drunk, regular, friends, wreck, apartm...|
|[opportunity, feeling, opportunities, motivation, build, selfconfidence, opportunity, existswhen,...|
|[comfort, zone, friends, ton, bar, club, comfortable, environment, alcoh

In [18]:
processed_data.columns

['id',
 'cleaned_text',
 'finished_unigrams',
 'finished_pos',
 'filtered_unigrams_by_pos']

In [19]:
#Now that POS was done, lemmatization makes more sense at this point

#Merge tokens as just one string to be able to take it as a document in the new Pipeline
from pyspark.sql import functions as F
tokens_as_string = F.udf(lambda x: ' '.join(x), T.StringType())
processed_data = processed_data.withColumn('joined_tokens', tokens_as_string(F.col('filtered_unigrams_by_pos')))

last_documentAssembler = DocumentAssembler() \
     .setInputCol('joined_tokens') \
     .setOutputCol('joined_document')

last_tokenizer = Tokenizer() \
     .setInputCols(['joined_document']) \
     .setOutputCol('tokenized')
     
lemmatizer = LemmatizerModel.load("../models/lemma_ewt_en_3.4.3_3.0_1651416655397/")\
      .setInputCols("tokenized")\
      .setOutputCol("lemmatized")

stopwords_cleaner = StopWordsCleaner() \
     .setInputCols(['lemmatized']) \
     .setOutputCol('final') \
     .setStopWords(stopwords)

last_finisher = Finisher() \
     .setInputCols(['final']) \

last_pipeline = Pipeline() \
     .setStages([last_documentAssembler,                  
                 last_tokenizer,
                 lemmatizer,
                 stopwords_cleaner,
                 last_finisher])

final_data = last_pipeline.fit(processed_data).transform(processed_data)


In [20]:
final_data.show(10, truncate=10)
                                                                                                                          

+-----+------------+-----------------+------------+------------------------+-------------+--------------+
|   id|cleaned_text|finished_unigrams|finished_pos|filtered_unigrams_by_pos|joined_tokens|finished_final|
+-----+------------+-----------------+------------+------------------------+-------------+--------------+
|hk5r2|  i had a...|       [appoin...|  [NN, NN...|              [appoin...|   appoint...|    [appoin...|
|iqimz|  i creat...|       [create...|  [VBN, N...|              [site, ...|   site se...|    [site, ...|
|pfzt5|  hello e...|       [recent...|  [RB, VB...|              [change...|   changes...|    [hope, ...|
|pk714|  i grew ...|       [grew, ...|  [VBD, N...|              [body, ...|   body dy...|    [body, ...|
|q0q8x|  i have ...|       [ask, c...|  [VB, NN...|              [conten...|   content...|    [conten...|
|q412v|  nothing...|       [opport...|  [NN, VB...|              [opport...|   opportu...|    [opport...|
|q5mqk|  im gett...|       [comfor...|  [NN, N

In [21]:
final_data.select('finished_final').show(50, truncate=100)

+----------------------------------------------------------------------------------------------------+
|                                                                                      finished_final|
+----------------------------------------------------------------------------------------------------+
|[appointment, dentist, last, reminder, email, reminder, phone, call, google, calendar, glance, ph...|
|[site, several, release, beta, forgive, useful, community, direction, future, development, fix, f...|
|[hope, vision, place, share, information, mindset, prosperity, mindset, money, purpose, attitude,...|
|[body, dysmorphia, disorders, social, anixety, depression, early, fat, weight, case, healthy, ave...|
|[content, process, stop, speak, hopeless, yearold, drunk, regular, wreck, apartment, romance, int...|
|[opportunity, feeling, opportunity, motivation, build, selfconfidence, opportunity, existswhen, c...|
|[comfort, zone, ton, bar, club, comfortable, environment, alcohol, noise

In [22]:
final_data.columns

['id',
 'cleaned_text',
 'finished_unigrams',
 'finished_pos',
 'filtered_unigrams_by_pos',
 'joined_tokens',
 'finished_final']

In [23]:
processed_data.unpersist()
final_data.persist()

DataFrame[id: string, cleaned_text: string, finished_unigrams: array<string>, finished_pos: array<string>, filtered_unigrams_by_pos: array<string>, joined_tokens: string, finished_final: array<string>]

## Vectorization

In [24]:
#Apply TF-IDF filtering
tfizer = CountVectorizer(inputCol='finished_final', outputCol='tf_features', minDF=0.01, maxDF=0.80)
tf_model = tfizer.fit(final_data)
tf_result = tf_model.transform(final_data)

idfizer = IDF(inputCol='tf_features', outputCol='tf_idf_features')
idf_model = idfizer.fit(tf_result)
tfidf_result = idf_model.transform(tf_result)

24/11/20 20:20:46 WARN BlockManager: Putting block rdd_30_9 failed due to exception org.apache.spark.SparkException: Failed to execute user defined function (HasSimpleAnnotate$$Lambda$3156/0x0000000801a2eb08: (array<array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>>) => array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>).
24/11/20 20:20:46 WARN BlockManager: Block rdd_30_9 could not be removed as it was not found on disk or in memory
24/11/20 20:20:46 WARN BlockManager: Putting block rdd_63_9 failed due to exception org.apache.spark.SparkException: Failed to execute user defined function (HasSimpleAnnotate$$Lambda$3156/0x0000000801a2eb08: (array<array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>>) => array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<str

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 47850)
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/software/spark-3.3.2-el8-x86_64/python/pyspark/sql/utils.py", line 190, in deco
    return f(*a, **kw)
  File "/software/spark-3.3.2-el8-x86_64/python/lib/py4j-0.10.9.5-src.zip/py4j/protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/software/spark-3.3.2-el8-x86_64/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/software/spark-3

Py4JError: py4j does not exist in the JVM

## LDA

In [ ]:
final_data.unpersist()
tfidf_result.persist()

In [ ]:
lda = LDA(k=15, maxIter=50, learningDecay=0.5, learningOffset = 50, featuresCol='tf_idf_features', seed=2503)
lda_model = lda.fit(tfidf_result)

In [ ]:
vocab = tf_model.vocabulary

def get_words(token_list):
     return [vocab[token_id] for token_id in token_list]
       
udf_to_words = F.udf(get_words, T.ArrayType(T.StringType()))

In [ ]:
num_top_words = 15

topics = lda_model.describeTopics(num_top_words).withColumn('topicWords', udf_to_words(F.col('termIndices')))
topics.select('topic', 'topicWords').show(truncate=90)

In [ ]:
lda_model.logPerplexity(tfidf_result)


In [ ]:
lda_model.logLikelihood(tfidf_result)